In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import torch
from torch_geometric.loader import DataLoader
import torch.nn as nn
from torch.utils.data import random_split
from tqdm import trange

In [2]:
hidden_channels = 256
class twoinone(nn.Module):
    
    def __init__(self):
        super().__init__()
        torch.manual_seed(42)
       
        self.nn1 = nn.Linear(100, hidden_channels*2)
        self.nn2 = nn.Linear(hidden_channels*2,hidden_channels*2)
        self.nn3 = nn.Linear(hidden_channels*2,hidden_channels*2)
        self.nn4 = nn.Linear(hidden_channels*2,hidden_channels*2)
        self.out = nn.Linear(hidden_channels*2, 50)
        
        self.loss_function = nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.0001, weight_decay=5e-4)
        self.counter=0
        self.progress = []
        
    def forward(self,x):
        
        x = x.to(device)
        x=self.nn1(x)
        x = x.relu()
        x=self.nn2(x)
        x = x.relu()
        x=self.nn3(x)
        x = x.relu()
        x=self.nn4(x)
        x = x.relu()
        out = self.out(x)
        
        return out
    
    def train(self,dataloader):
        losses=[]
        for i in range(len(dataloader)):
            data = dataloader[i]
            outputs = self.forward(data.x.float())
            y = data.y.to(device)
            y = y.float()
            loss_t = self.loss_function(outputs.to(torch.float32),y.to(torch.float32))
            losses.append(loss_t)
        loss = torch.stack(losses,dim=0).mean(dim=0)        
        #outputs = self.forward(data.x.float())
        #y = data.y.to(device)
        #y = y.float()
        #loss = self.loss_function(outputs.to(torch.float32),y.to(torch.float32))
       
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss
    
    def val(self,dataloader):
        losses=[]
        for i in range(len(dataloader)):
            data = dataloader[i]
            outputs = self.forward(data.x.float())
            y = data.y.to(device)
            y = y.float()
            loss_t = self.loss_function(outputs.to(torch.float32),y.to(torch.float32))
            losses.append(loss_t)
        loss = torch.stack(losses,dim=0).mean(dim=0)
        return loss
    
    def pred(self,data):
        output = self.forward(data.x.float())
        return output

In [3]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
model = twoinone()
model.to(device)

twoinone(
  (nn1): Linear(in_features=100, out_features=512, bias=True)
  (nn2): Linear(in_features=512, out_features=512, bias=True)
  (nn3): Linear(in_features=512, out_features=512, bias=True)
  (nn4): Linear(in_features=512, out_features=512, bias=True)
  (out): Linear(in_features=512, out_features=50, bias=True)
  (loss_function): MSELoss()
)

In [4]:
data = torch.load("/home/chengc/workspace/cc/0815/qm_9_2in1.pt")
train_data,val_data,test_data = random_split(dataset = data,lengths = [100000,20000,7468],generator = torch.Generator().manual_seed(42))
train_loader = DataLoader(train_data, batch_size = 64, shuffle = True)
val_loader = DataLoader(val_data, batch_size = 64, shuffle = True)

In [15]:
thresh = 500
model_path = "/home/chengc/workspace/cc/0815/model/qm_9_2in1_model.pth"
model_path_ckp = "/home/chengc/workspace/cc/0815/model/qm_9_2in1_model_ckp.pth"
for epoch in trange(2000):
    losses=[]
    for loader in train_loader:
        loss = model.train(loader)
        losses.append(loss)
    lossmean = torch.stack(losses,dim=0).mean(dim=0)
    #print(lossmean)
    vlosses=[]
    for Vloader in val_loader:
        vloss = model.val(Vloader)
        vlosses.append(vloss)
    vlossmean = torch.stack(vlosses,dim=0).mean(dim=0)
    #print(vlossmean)
    if vlossmean < thresh:
    #and lossmean < 0.1:
        print(f"Save Model : Epoch {epoch} | Trainloss {lossmean}| Valiloss {vlossmean}")
        torch.save(model.state_dict(),model_path)
        thresh = vlossmean
    if (epoch % 100 == 0):
        print(f"Epoch {epoch} | Trainloss {lossmean}| Valiloss {vlossmean}")
        torch.save(model.state_dict(),model_path_ckp)

  0%|                                                                                                                                                | 1/2000 [04:04<136:01:06, 244.96s/it]

Save Model : Epoch 0 | Trainloss 4.895115375518799| Valiloss 4.592116832733154
Epoch 0 | Trainloss 4.895115375518799| Valiloss 4.592116832733154
Save Model : Epoch 1 | Trainloss 2.7575457096099854| Valiloss 4.522714614868164


  0%|▏                                                                                                                                               | 2/2000 [08:07<135:11:10, 243.58s/it]

Save Model : Epoch 2 | Trainloss 2.667605400085449| Valiloss 4.478421211242676


  0%|▏                                                                                                                                               | 3/2000 [12:20<137:32:14, 247.94s/it]

...

Epoch 500 | Trainloss 1.1258894205093384| Valiloss 4.886943340301514


 30%|█████████████████████████████████████████▊                                                                                                 | 601/2000 [42:32:44<103:18:54, 265.86s/it]

Epoch 600 | Trainloss 1.0490895509719849| Valiloss 5.0810346603393555


 32%|████████████████████████████████████████████▍                                                                                               | 634/2000 [45:04:08<97:06:17, 255.91s/it]


KeyboardInterrupt: 

In [ ]:
#Save Model : Epoch 48 | Trainloss 2.287713050842285| Valiloss 4.337092399597168